# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
import pandas as pd
df=pd.read_csv('/datasets/users_behavior.csv')# открываем файл
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


Описание данных

-сalls — количество звонков

-minutes — суммарная длительность звонков в минутах

-messages — количество sms-сообщений

-mb_used — израсходованный интернет-трафик в Мб

is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).-

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [3]:
df['calls']=df['calls'].astype('int')# Изменим данные в количестве звонков на целочисленный.
df['messages']=df['messages'].astype('int')# Изменим данные в количестве сообщений на целочисленный.

Пропущенных значений нет.Проверим на дубликаты.

In [4]:
df.duplicated().sum()

0

Итак, в данных нет дубликатов,все значения заполнены,все данные имеют нужный тип. Можно начинать обучение.

## Разбейте данные на выборки

Разобьем данные по правилу 70-20-10 (70% для обучения, 20% для проверки, 10% для тестирования).

In [5]:
# Загрузим необходимую библиотеку.
from sklearn.model_selection import train_test_split
# Разобьем сначала на обучающее и проверочное 
df_train, df_test = train_test_split(df, test_size=0.3, random_state=12345)
features = df_test.drop('is_ultra', axis=1)
target = df_test['is_ultra']
# Теперь проверочную выборку разобьем на валидационную и тестовую
features_valid, features_test, target_valid, target_test = train_test_split(features, target, test_size=0.35, random_state=12345)
features_train = df_train.drop('is_ultra', axis=1)
target_train = df_train['is_ultra']
print("Размер тренировочной выборки", features_train.shape)
print("Размер валидационной выборки",features_valid.shape)
print("Размер тестовой выборки",features_test.shape)


Размер тренировочной выборки (2249, 4)
Размер валидационной выборки (627, 4)
Размер тестовой выборки (338, 4)


## Исследуйте модели

1.Модель логистической регрессии. В данной модели будем использовать следущие гиперпараметры:random_state(для постоянства результата равный 12345).solver='lbfgs'(алгоритм, который будет строить модель. Алгоритм 'lbfgs' — один из самых распространённых. Он подходит для большинства задач), max_iter (задаётся максимальное количество итераций обучения)

Поскольку перед нами задача классификации(«Ультра» — 1, «Смарт» — 0) во всех моделях мы будем использовать метрику качества Accurancy

In [6]:
#Загрузим необходимую библиотеку
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='liblinear',max_iter=1000)
model.fit(features_train, target_train)
accuracy = model.score(features_valid, target_valid)
print("Качество модели:", accuracy)

Качество модели: 0.6953748006379585


2.Модель дерево решения. Здесь в качестве гиперпарамеиров используем random_state и переберем различную глубину дерева решений для нахождения параметра модели с самым высоким параметром точности.

In [7]:
from sklearn.tree import DecisionTreeClassifier
best_accuracy = 0
best_depth = 0
best_sample_leaf = 0
best_sample_split = 0
for depth in range(1,15):
    for sample_leaf in range (1,15):
        for sample_split in range (2,15):
            model = DecisionTreeClassifier(max_depth=depth,min_samples_leaf = sample_leaf,min_samples_split=sample_split,random_state=12345)
            model.fit(features_train, target_train)
    accuracy = model.score(features_valid, target_valid)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_depth = depth
        best_sample_leaf = sample_leaf
        best_sample_split = sample_split
print("Глубина дерева:", best_depth,"Sample_leaf:",best_sample_leaf,"sample_split",best_sample_split ," Качество модели:", best_accuracy)

Глубина дерева: 8 Sample_leaf: 14 sample_split 14  Качество модели: 0.7942583732057417


3.Модель случайный лес. Здесь гиперпараметрами выступят:random_state=12345(Зафиксируем псевдослучайность).

n_estimators - число деревьев в случайном лесу.Чем их больше,тем лучше обучение, но увеличение может сказаться на производительности. По умолчанию равен 100,но с учетом использования еще одного гиперпараметра ,увеличивающимй время расчета, снизим это значение до 25.

max_depth - максимальная глубина этих деревьев.Здесь также при увеличении глубины возрастает качество и время расчета.По умолчанию параметр равен None.

In [8]:
from sklearn.ensemble import RandomForestClassifier
best_accuracy = 0
best_depth = 0
best_sample_split = 0

for depth in range(1,25):
    for est in range(1,25):
        for sample_split in range (2,5):
            model = RandomForestClassifier(max_depth=depth, n_estimators=est,min_samples_split=sample_split,random_state=12345)                              
            model.fit(features_train, target_train)
            accuracy = model.score(features_valid, target_valid)
        if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_depth = depth
                best_est = est
                best_sample_split = sample_split
                
print("Глубина дерева:", best_depth, "Количество деревьев:",best_est,"Количество выборок для разделения модели:",best_sample_split, "Качество модели:", best_accuracy)

Глубина дерева: 8 Количество деревьев: 11 Количество выборок для разделения модели: 4 Качество модели: 0.8165869218500797


По итогам обучения на 2х моделях удалось достичь уровня точности выше 75. Это модели дерева решений и случайного леса.Проведем проверку на тестовой выборке c учетом параметров моделей самой высокой точности.

## Проверьте модель на тестовой выборке

In [9]:
#Модель логистической регрессии
model = LogisticRegression(solver='lbfgs',max_iter=1000)
model.fit(features_train, target_train)
accuracy = model.score(features_test, target_test)
print("Качество модели логистической регрессии:", accuracy)
#Модель дерева решений
model = DecisionTreeClassifier(max_depth=3,min_samples_leaf =14,min_samples_split=14, random_state=12345)
model.fit(features_train, target_train)
accuracy = model.score(features_test, target_test)
print("Качество модели дерева решений:", accuracy)
#Модель случайного леса
model = RandomForestClassifier(max_depth=8, n_estimators=11,min_samples_split=4,random_state=12345)
model.fit(features_train, target_train)
accuracy = model.score(features_test, target_test)
print("Качество модели случайного леса:", accuracy)

Качество модели логистической регрессии: 0.6982248520710059
Качество модели дерева решений: 0.7781065088757396
Качество модели случайного леса: 0.8076923076923077


На тестовой выборке также максимальная точность у модели случайного леса с показателем 0.804.

## (бонус) Проверьте модели на адекватность

Для проверки адекватности воспользуемся встроенным модулем DummyClassifier, который построит простейшую (dummy)  модель, которая всегда предсказывает наиболее часто встречающийся класс(для этого мы используем стратегию 'most_frequent').Мы получим Accurancy для сравнительной оценки построенной  модели.

In [10]:
from sklearn.dummy import DummyClassifier
dc = DummyClassifier(strategy='most_frequent', random_state=12345)
dc.fit(features_train, target_train)
score_dc = dc.score(features_valid,  target_valid)
print('DummyClassifier accurancy:', score_dc)

DummyClassifier accurancy: 0.6985645933014354


Точность модели DummyClassifier ниже, чем у построенной модели случайного леса модель адекватна и эффективна.

## Выводы


-В ходе исследования нами были построены и проверены на точность 3 модели: Decision Tree Classifier, RandomForestClassifier и Logistic Regression.

-Самой точной оказалась модель RandomForestClassifier с гиперпарамеирами max_depth = 8, n_estimators = 7. Accurancy на тестовой выборке составила 0.804.

- Также была построена базовая модель для проверки вменяемости с помощью DummyClassifier. Accurancy этой модели составила 0.69,что подтвердило качество построенной модели. 